# UR5 Simulation Notebook

## Overview

In this notebooks is described how to actually simulate and control a simulated UR5 robot in ROS.

## 1. GAZEBO SIMULATION SETUP
--------------------------------

To simulate a robot two things are necessary: a realistic model of the robot and a set of controllers that move the individual joints. 

Requisites:

* Check these packages are installed:
```bash
sudo apt-get install ros-kinetic-gazebo_ros_control
sudo apt-get install ros-kinetic-controller-manager
sudo apt-get install ros-kinetic-joint-state-controller
sudo apt-get install ros-kinetic-joint-state-publisher
sudo apt-get install ros-kinetic-joint-state-controller
sudo apt-get install ros-kinetic-moveit
```

### 1.1. Custom URDF

The simulation of a robtot always starts with a URDF (Universal Robot Description Format), the native format for describing robots in ROS. The URDF is used to create the 3D model for the 3D visualization.

The description of only the robot arm is *usually* provided by the manifacturer, like the [UR5](https://github.com/ros-industrial/universal_robot). However is usually necessary to createa  customized robot descriptor: to add a gripper, camera, moving base or maybe a robot with two arms!

This is the visualization of the customized UR5 robot URDF wich uses the gripper from the **gripper description** package.

<p align="center">
     <img src="img/gripper_custom.png" width="320">
</p>

* **Using Xacro** to include urdf in an other urdf makes easier to build the robot description block by block.

```xml
<?xml version="1.0"?>
<robot xmlns:xacro="http://wiki.ros.org/xacro" name="ur5_arm">

    <!--XACRO ARGUMENTS-->
    <xacro:arg name="prefix" default=""/>

    <!--XACRO INCLUDES-->
    <xacro:include filename="$(find gripper_description)/urdf/gripper.urdf.xacro" />
    <xacro:include filename="$(find ur_description)/urdf/ur5_robot.urdf.xacro" />

    <!--ROBOT LAST LINK TO GRIPPER BASE LINK-->
    <joint name="gripper_joint" type="fixed">
        <parent link="ee_link"/>
        <child link="gripper_link"/>
        <origin xyz="0.0435 0 0" rpy="0 0 0"/>
        <axis xyz="0 1 0" />
    </joint>
    
    <!--GRIPPER URDF as MACRO-->
    <xacro:gripper prefix="$(arg prefix)"/>
    
    <!--UR5 URDF as MACRO-->
    <xacro:ur5 prefix="$(arg prefix)"/>

</robot>
```

The **Prefix** is a simple argument wich can be useful in future (e.g. right and left arm)

* **Add Inertia elements** to all links of the robot URDF: Inertia is **required** to simulate the physics of the robot. 

```xml
<inertial>
<origin xyz="0 0 0" rpy="0 0 0"/>
<mass value="0.3"/>
<inertia ixx="0.0001" ixy="0" ixz="0" iyy="0.0002" iyz="0" izz="0.0001" />
</inertial>
```

If the simulation is started at this point, the robot will just fall as there is no force applied to the joints to mantain their position; the robot requires **controller**.

### 1.2. Joints Controllers

It is necessary to setup simulated controllers in order to actuate the joints of the robot (and the gripper) in the simulation, the UR5 was already configured by the vendor. 

Here is how the gripper was configured to be used in simulation:

* **Add transmission elements to the URDF**: the **<transmission>** element is used to link actuators to joints. These are defined in a separated .xacro file and then included in the main urdf. 

```xml
  <transmission name="${prefix}left_finger_transmission">
    <type>transmission_interface/SimpleTransmission</type>
    <joint name="${prefix}left_gripper_finger_joint">
      <hardwareInterface>hardware_interface/EffortJointInterface</hardwareInterface>
    </joint>
    <actuator name="${prefix}eft_finger_motor">
      <hardwareInterface>hardware_interface/EffortJointInterface</hardwareInterface>
      <mechanicalReduction>1</mechanicalReduction>
    </actuator>
  </transmission>

  <transmission name="${prefix}right_finger_transmission">
    <type>transmission_interface/SimpleTransmission</type>
    <joint name="${prefix}right_gripper_finger_joint">
      <hardwareInterface>hardware_interface/EffortJointInterface</hardwareInterface>
    </joint>
    <actuator name="${prefix}right_finger_motor">
      <hardwareInterface>hardware_interface/EffortJointInterface</hardwareInterface>
      <mechanicalReduction>1</mechanicalReduction>
    </actuator>
  </transmission>
```

The two finger joints of the Gripper from the URDF are linked to two actuators using **EffortJointInterface**
 
* **Add the gazebo_ros_control plugin**: In addition to the transmission tags, a Gazebo plugin is to be added to the URDF that parses the transmission tags and loads the appropriate hardware interfaces and controller manager. The namespace is important as will be used by the topics spawned by the controllers.

```xml
<gazebo>
  <plugin name="gazebo_ros_control" filename="libgazebo_ros_control.so">
    <robotNamespace>{$prefix}gripper</robotNamespace>
  </plugin>
</gazebo>
```
 
* **Create a .yaml config file**: The PID gains and controller settings must be saved in a yaml file that gets loaded to the param server via the roslaunch file.

```yaml
gripper_controller:
  type: "effort_controllers/JointTrajectoryController"
  joints:
    - right_gripper_finger_joint
    - left_gripper_finger_joint
  gains:
    right_gripper_finger_joint: {p: 100, i: 1, d: 10, i_clamp: 1.0}
    left_gripper_finger_joint: {p: 100, i: 1, d: 10, i_clamp: 1.0}
  constraints:
    goal_time: 3.0
    right_gripper_finger_joint:
      goal: 0.02
    left_gripper_finger_joint:
      goal: 0.02
```

* **Create a roslaunch file**: Create a roslaunch *contoller_utils.launch* file for starting the ros_control controllers. 

```xml
<launch>

  <!-- Load rosparam from the controllers .yaml -->
  <rosparam file="$(find ur_gazebo)/controller/joint_state_controller.yaml" command="load"/>
  
  <!-- Load joint controller configurations from YAML file to parameter server -->
  <rosparam file="$(find ur5_arm_gazebo)/controller/arm_controller.yaml" command="load"/>

  <!-- load the controllers -->
  <node name="arm_controller_spawner" pkg="controller_manager" type="controller_manager" args="spawn arm_controller gripper_controller joint_state_controller" respawn="false" output="screen"/>

  <!-- Robot state publisher -->
  <node pkg="robot_state_publisher" type="robot_state_publisher" name="robot_state_publisher">
    <param name="publish_frequency" type="double" value="50.0" />
    <param name="tf_prefix" type="string" value="" />
  </node>
    
</launch>
```

Explanation:

The tag "rosparam" loads the controller settings to the parameter server by loading a .yaml configuration file.

The **controller_spawner** node starts the joint position controllers for the ur5 arm and gripper by running a python script that makes a service call to the ros_control controller manager. It also loads a third controller that publishes the joint states of all the joints with hardware_interfaces and advertises the topic on /joint_states.

The final line starts a **robot_state_publisher** node that simply listens to */joint_states messages* from the joint_state_controller then publishes the **frame transforms** to /tf topic. This allows to see your simulated robot in Rviz move as well.

### 1.3 Starting a Base Simulation.
    
To start a simple simulation of only the robot arm in a *empty world* a launchfile is created:
    
```xml
<launch>
  <arg name="paused" default="false" doc="Starts gazebo in paused mode" />
  <arg name="gui" default="true" doc="Starts gazebo gui" />
  
  <!-- startup simulated world -->
  <include file="$(find gazebo_ros)/launch/empty_world.launch">
    <arg name="world_name" default="worlds/empty.world"/>
    <arg name="paused" value="$(arg paused)"/>
    <arg name="gui" value="$(arg gui)"/>
  </include>

  <!-- send robot urdf to param server -->
  <param name="robot_description" command="$(find xacro)/xacro --inorder '$(find ur5_arm)/urdf/ur5_arm.urdf.xacro'" />

  <!-- spawn robot in gazebo -->
  <node name="spawn_gazebo_model" pkg="gazebo_ros" type="spawn_model" args="-urdf -param robot_description -model robot -x 0.0 -y 0.0 -z 0.1" respawn="false" output="screen" />

  <!-- launch gazebo controllers -->
  <include file="$(find ur5_arm_gazebo)/launch/controller_utils.launch"/>

</launch>
```

Explanation:

The launcfile does three things: starts an empty gazebo world, then the robot URD is loaded as the **robot_description** param in the parameter server: the robot_description is used to spawn the robot in the simulated world, finally the **joints_controllers** previously defined are loaded, these will allow to send commands to the simulator.
    
If everithing is correct, after launching the simulation the following topics will be pubished by gazebo:
    
<p align="center">
     <img src="img/gazebo_topics.png" width="480">
</p>

These are discussed next.

### 1.3. (Optional) Gazebo Grasp Plugin
The [Gazebo grasp plugin](https://github.com/JenniferBuehler/gazebo-pkgs) fixes an object which is grasped to the robot hand to avoid problems with physics engines and to help the object staying in the robot hand without slipping out. The implementation is a model plugin,wich is loaded from the robot URDF:

```xml
   <gazebo>
     <plugin name="gazebo_grasp_fix" filename="libgazebo_grasp_fix.so">
       <arm>
         <arm_name>ur5_arm</arm_name>
         <palm_link>ee_link</palm_link>
         <gripper_link>right_gripper_finger_link</gripper_link>
         <gripper_link>left_gripper_finger_link</gripper_link>
       </arm>
       <forces_angle_tolerance>100</forces_angle_tolerance>
       <update_rate>4</update_rate>
       <grip_count_threshold>4</grip_count_threshold>
       <max_grip_count>8</max_grip_count>
       <release_tolerance>0.001</release_tolerance>
       <disable_collisions_on_attach>false</disable_collisions_on_attach>
       <contact_topic>__default_topic__</contact_topic>
     </plugin>
   </gazebo>
```

### 2. Interfacing with the simulation
------------------

#### 2.1. Retrieve the Joints Status

#### 2.2. Send Commands to Joints Controllers

To send a command to the **gripper_controller**:

```bash
rostopic pub /gripper_controller/command trajectory_msgs/JointTrajectory "header:
  seq: 0
  stamp:
    secs: 0
    nsecs: 0
  frame_id: ''
joint_names: [right_gripper_finger_joint,left_gripper_finger_joint]
points:
- positions: [0.1,0.1]
  velocities: [1,1]
  accelerations: [1,1]
  effort: [1,1]
  time_from_start: {secs: 1, nsecs: 0}" 

```

### 3. ROS MOVEIT! package
------------------------------

[source: the construct](http://www.theconstructsim.com/ros-movelt/)

**MoveIt!** ia a motion planning framework.

Moving the arm to achieve a desired position is a non-trivial task: it requires to produce the sequence of values that every joint of the arm must follow (in coordination with the other joints), so the end effector moves from its current place to the desired place. 

This task is called **motion planning**. The result of a motion planning is the sequence of movements that all the joints of the arm have to perform in order to move from current location to the desired one.

MoveIt! can also **execute** the plan in the robot.

In order to be able to use moveit_ros with any robot, first, some configuration steps are necessary, **the only prerequisite is the appropriate URDF**.

### 3.1 Creation of a Moveit! Configuration Package
-------------------------

A graphical tool is provided to create a configuration package:

```bash
roslaunch moveit_setup_assistant setup_assistant.launch
```

The steps are:

#### 3.1.1 START -> Load the xacro file

<p align="center">
     <img src="img/moveit_config1.png" width="480">
</p>

This loads the URDF and identifies all joints of the robot.

#### 3.1.1 SELF-COLLISIONS -> Generate Self-Collision Matrix

<p align="center">
     <img src="img/moveit_config2.png" width="480">
</p>

This checks for links that would collide during trajectories.

#### 3.1.3 VIRTUAL JOINTS ->Define Virtual Joint 

<p align="center">
     <img src="img/moveit_config3.png" width="480">
</p>

This virtual joint attaches the robot to the simulated world.

#### 3.1.4 PLANNING GROUPS -> Add Group

A Planning Group combines different parts of the robot for Moveit! to control as one. A group for the arm and one for the gripper should be created.

The planning group of the ur5 is called **arm** and it will use the *KDL kinematics plugin* as solver and *RRT* as planner:

<p align="center">
     <img src="img/moveit_config4.png" width="480">
</p>

Add the joints that belong to that planning group: the names can be found again in the URDF file

<p align="center">
     <img src="img/moveit_config5.png" width="480">
</p>

The planning group of the gripper is called **hand**, and it doesnt require planners or solvers:

<p align="center">
     <img src="img/moveit_config6.png" width="480">
</p>

Then, Add the joints that belong to that planning group: the names can be found again in the URDF file

<p align="center">
     <img src="img/moveit_config7.png" width="480">
</p>

The final result should look like this:

<p align="center">
     <img src="img/moveit_config8.png" width="480">
</p>

#### 3.1.4 ROBOT POSES -> Add Pose

Some Predefined Poses are created: these are usefull to move the robot to starting position and opening and closing the gripper.

<p align="center">
     <img src="img/moveit_config9.png" width="480">
</p>

<p align="center">
     <img src="img/moveit_config10.png" width="480">
</p>

#### 3.1.5 END EFFECTORS --> Add End Effector

The End effector group has to be the one containg the gripper joints, while the parent link is the link to wich the gripper URDF has been attached.

<p align="center">
     <img src="img/moveit_config10.png" width="480">
</p>

#### 3.1.6 AUTHOR INFORMATION

These are mandatory to create the package

#### 3.1.7 CONFIGURATION FILES --> Generate Package

Creates the package and all configuration files. Create or select an appropriate folder in the catkin workspace named myrobot_moveit_config.

### 3.2 Connectf Moveit! To the Actual Robot
-------------------------

A demo can be started running:

```bash
roslaunch ur5_arm_moveit_config demo.launch
```

<p align="center">
     <img src="img/moveit_demo.png" width="480">
</p>

However, this demo does not control yet the actual robot, to do so some extra configuration files are required to interface moveit with the **JointTrajectory action servers**, so the planned trajectories can be sent to the robot.

To do so:

* Create a file called **controllers.yaml** under the **/config** directory with the following content:

```yaml
controller_list:
  - name: arm_controller
    action_ns: "follow_joint_trajectory"
    type: FollowJointTrajectory
    joints:
      - shoulder_pan_joint
      - shoulder_lift_joint
      - elbow_joint
      - wrist_1_joint
      - wrist_2_joint
      - wrist_3_joint
  - name: gripper_controller
    action_ns: "follow_joint_trajectory"
    type: FollowJointTrajectory
    joints: 
      - left_gripper_finger_joint
      - right_gripper_finger_joint
```

* Create a file called **joint_names.yaml** under the **/config** directory with the following content:

```yaml
controller_joint_names: [shoulder_pan_joint, shoulder_lift_joint, elbow_joint, wrist_1_joint, wrist_2_joint, wrist_3_joint,left_gripper_finger_joint,right_gripper_finger_joint]
```

* Create a file called **ur5_arm_moveit_controller_manager.launch.xml** under the **/launch** directory with the following content:

```xml
<launch>
  <rosparam file="$(find ur5_arm_moveit_config)/config/controllers.yaml"/>
  <param name="use_controller_manager" value="false"/>
  <param name="trajectory_execution/execution_duration_monitoring" value="false"/>
  <param name="moveit_controller_manager" value="moveit_simple_controller_manager/MoveItSimpleControllerManager"/>
</launch>
```

* Create a file called **ur5_arm_planning_execution.launch** under the **/launch** directory with the following content:

```xml
<launch>
 
  <rosparam command="load" file="$(find ur5_arm_moveit_config)/config/joint_names.yaml"/>
 
  <include file="$(find ur5_arm_moveit_config)/launch/planning_context.launch" >
    <arg name="load_robot_description" value="true" />
  </include>
 
  <node name="joint_state_publisher" pkg="joint_state_publisher" type="joint_state_publisher">
    <param name="/use_gui" value="false"/>
    <rosparam param="/source_list">[/joint_states]</rosparam>
  </node>
 
  <include file="$(find ur5_arm_moveit_config)/launch/move_group.launch">
    <arg name="publish_monitored_planning_scene" value="true" />
  </include>
 
  <include file="$(find ur5_arm_moveit_config)/launch/moveit_rviz.launch">
    <arg name="config" value="true"/>
  </include>
 
</launch>
```

### 3.3 
-------------------------
https://www.youtube.com/watch?v=ySceuKCS5mE